# <center><font color="green"> https://bit.ly/ptpjb-2021-12</font></center>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://bit.ly/ptpjb-2021-12)

# <center><font color="blue">12 - Advanced prediction in Machine Learning</font></center>

<center><img alt="" src="images/cover_ptpjb_2021.png"/></center> 

## <center><font color="blue">tau-data Indonesia</font><br>(C) Taufik Sutanto - 2021</center>
<center><a href="https://tau-data.id">https://tau-data.id</a> ~ <a href="mailto:taufik@tau-data.id">taufik@tau-data.id</a></center>

# <center><font color="blue"> Outline: Model Klasifikasi Lanjutan</font></center>

* Support Vector Machines
* Evaluasi revisited: Underfitting & Overfitting
* Pipelining & Parameter Optimization
* Proper Model Selection
* Ensemble Learning
* Imbalance Learning
* Studi Kasus

<img src="images/meme-cartoon/meme model on train and test data.jpg" height="250" />

In [ ]:
# Loading Modules
import warnings; warnings.simplefilter('ignore')
import pickle
import pandas as pd, matplotlib.pyplot as plt
import time, numpy as np, seaborn as sns
from sklearn import svm, preprocessing
from sklearn import  tree
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import make_pipeline 
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from tqdm import tqdm
sns.set(style="ticks", color_codes=True)
"Done"

In [ ]:
# Mulai dengan Load Data dari Modul sebelumnya terlebih dahulu
file_ = "data/data_Module-11.pckl"
try: # Running Locally, yakinkan "file_" berada di folder "data"
    f = open(file_, 'rb')
    data = pickle.load(f); f.close()
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/ptpjb/master/{file_}
    f = open(file_, 'rb')
    data = pickle.load(f); f.close()

df_, df1, y1, df2A, df2B, y2 = data
df_.shape, df_.keys()

In [ ]:
# Akan sama dengan module sebelumnya karena nilai SEED sama.
df1_train, df1_test, y1_train, y1_test = train_test_split(df1, y1, test_size=0.3, random_state=33)
df2A_train, df2A_test, y2_train, y2_test = train_test_split(df2A, y2, test_size=0.3, random_state=33) #No One-Hot
df2B_train, df2B_test, y2_train, y2_test = train_test_split(df2B, y2, test_size=0.3, random_state=33) # One-Hot
"Done"

# <center><font color="blue">Support Vector Machine (SVM)</font></center>

Misal data dinyatakan sebagai berikut:
$\{(\bar{x}_1,y_1),...,(\bar{x}_n,y_n)\}$, dimana $\bar{x}_i$ adalah
input pattern untuk data ke $i^{th}$ dan $y_i$ adalah nilai target yang diinginkan. Kategori
(class) direpresentasikan dengan $y_i=\{-1,1\}$. Sebuah **bidang datar (hyperplane)** yang memisahkan kedua kelas ini ("linearly separable") adalah:
$$ \bar{w}'\bar{x} + b=0 $$
dimana $\bar{x}$ adalah input vector (prediktor), $\bar{w}$ weight, dan $b$ disebut sebagai bias.

<p><img src="images/Pemodelan_SVM_.png" alt="" width="334" height="360" /></p>

# <center><font color="blue">Kelebihan Pemodelan SVM</font></center>

<img alt="" src="images/hard_margin_svm.png" style="width: 400px; height: 181px;" />

# <center><font color="blue">Support Vector Machine: Soft Margin</font></center>
<p><img style="undefined: undefined;" src="images/6_SVM.jpg" alt="" /> <img style="undefined: undefined;" src="images/svm_opt.png" alt="" width="461" height="163" /></p>
<ul>
<li>Diselesaikan dengan "mudah" via linear/quadratic programming.</li>
<li>Fungsi ini **Convex** sehingga penyelesaiannya menghasilkan nilai Global Optimal.</li>
</ul>

* **Interpretasi**: Recursive Feature Elimination (RFE) method melihat bentuk kuadrat dari setiap komponen *w* (higher better).

# <center><font color="blue">SVM Kernel (trick): $R^m \rightarrow R^n, n\geq m$</font></center>

<p><img alt="" src="images/6_SVM_Kernel.jpg" style="height:168px; width:306px" /></p>

# <center><font color="blue">Contoh Fungsi Kernel</font></center>

* Misal x = (x1, x2, x3); y = (y1, y2, y3). 
* dan fungsi pemetaan variabelnya f(x) = (x1², x1x2, x1x3, x2x1, x2², x2x3, x3x1, x3x2, x3²), 
* maka kernelnya adalah K(x, y ) = <f(x), f(y)> = <x, y>².
* Contoh numerik misal x = (1, 2, 3) dan y = (4, 5, 6). maka:
* f(x) = (1, 2, 3, 2, 4, 6, 3, 6, 9) <br> f(y) = (16, 20, 24, 20, 25, 30, 24, 30, 36)
* <f(x), f(y)> = 16 + 40 + 72 + 40 + 100+ 180 + 72 + 180 + 324 = 1024
* complicated!... **Menggunakan fungsi kernel perhitungannya bisa disederhanakan**:
* K(x, y) = (4 + 10 + 18)² = 32² = 1024
* Artinya perhitungan di dimensi yang tinggi dapat dilakukan di dimensi satu via inner product!.

# <center><font color="blue">Contoh Fungsi Kernel yang Populer</font></center>

<p><img alt="" src="images/Well-Known_Kernels.png" style="width: 400px; height: 208px;" /></p>

# <center><font color="blue">Kelebihan dan Kekurangan SVM</font></center>

<p><b>Pros</b></p>

<ul>
	<li>Akurasinya Baik</li>
	<li>Bekerja dengan baik untuk sampel data yang relatif kecil</li>
	<li>Hanya bergantung pada SV ==&gt; meningkatkan efisiensi</li>
	<li>Convex ==&gt; Minimum Global ==&gt; Pasti Konvergen</li>
</ul>

<p><b>Cons</b></p>

<ul>
	<li>Tidak efisien untuk data yang besar</li>
	<li>Akurasi terkadang rendah untuk multiklasifikasi (sulit mendapatkan hubungan antar kategori di modelnya)</li>
	<li>Tidak robust terhadap noise</li>
</ul>

Bacaan lebih lanjut:
* https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-theory-f0812effc72
* Contoh Perhitungan Manual: https://slideplayer.info/slide/3672979/?fbclid=IwAR3Tteg_PbKwkBxV63FGfat3o9UBfHBnjvGHwlyYcrxKTWeb6gfsSpBAQBE


In [ ]:
# Fitting and evaluate the model
dSVM = svm.SVC(C = 10**5, kernel = 'linear')#Misal menggunakan kernel Linear

dSVM.fit(df1_train, y1_train)
y_SVM1 = dSVM.predict(df1_test)

print(confusion_matrix(y1_test, y_SVM1))
print(classification_report(y1_test, y_SVM1))

In [ ]:
# The Support Vectors
print('index dr SV-nya: ', dSVM.support_)
print('Vector Datanya: \n', dSVM.support_vectors_)

In [ ]:
# Model Weights for interpretations
print('w = ',dSVM.coef_)
print('b = ',dSVM.intercept_)

In [ ]:
# Menggunakan Kernel: http://scikit-learn.org/stable/modules/svm.html#svm-kernels
for kernel in ('sigmoid', 'poly', 'rbf', 'linear'):
    dSVM = svm.SVC(kernel=kernel)
    dSVM.fit(df1_train, y1_train)
    y_SVM = dSVM.predict(df1_test)
    print(accuracy_score(y1_test, y_SVM))

In [ ]:
dSVM = svm.SVC(C = 10**5, kernel = 'linear')
mulai = time.time()
scores_svm = cross_val_score(dSVM, df1, y1, cv=10) # perhatikan sekarang kita menggunakan seluruh data
waktu = time.time() - mulai
print("Accuracy SVM: %0.2f (+/- %0.2f), Waktu = %0.3f detik" % (scores_svm.mean(), scores_svm.std() * 2, waktu))

In [ ]:
# Visualisasi untuk mengevaluasi & membandingkan model dengan lebih baik lagi
df_['SVM'] = scores_svm
p = sns.boxplot(data = df_)
df_.min()

# <center><font color="blue">Induktif Bias</font></center>

<ul>
	<li>Bias penaksiran parameter (statistik)</li>
	<li>Induktif Bias Sample (Machine Learning - Tom Mitchel)</li>
	<li>Induktif Bias Pemilihan Classifier (Statistical Learning Theory - Vapnik)</li>
</ul>
<img alt="" src="images/inductive_biases_.png" style="width: 600px; height: 153px;" />

<img alt="" src="images/class.png" />

# <center><font color="blue">(Hyper)Parameter Optimization</font></center>

* Perbandingan yang baru saja kita lakukan walau sudah CV, namun belum sepenuhnya valid.
* Saat membandingkan model, maka kita harus meyakinkan seluruh model mendapatkan parameternya yang optimal.

<img alt="" src="images/hyper-parameter_tunning.png" />

In [ ]:
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
file = 'data/diabetes_data.csv'

try:
    # Local jupyter notebook, assuming "file" is in the "data" directory
    data = pd.read_csv(file, names=names)
except:
    # it's a google colab... create folder data and then download the file from github
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/{file}
    data = pd.read_csv(file, names=names)
    
print(data.shape, set(data['class']))
data.sample(5)

In [ ]:
# Split Train-Test

X = data.values[:,:8]  # Slice data (perhatikan disini struktur data adalah Numpy Array)
Y = data.values[:,8]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=99)

print(set(Y), x_train.shape, x_test.shape, sep=', ')

# <center><font color="blue">Kita Jalankan Terlebih Dahulu dengan "Default Parameter"</font></center>

In [ ]:
clf = LogisticRegression(solver='liblinear')
kNN = neighbors.KNeighborsClassifier()
gnb = GaussianNB()
dt = tree.DecisionTreeClassifier()
rf = RandomForestClassifier()
svm_ = svm.SVC()

Models = [('Regresi Logistik', clf), ('k-NN',kNN), ('Naive Bayes',gnb), ('Decision Tree', dt), ('Random Forest', rf), ('SVM', svm_)]
Scores = {}
for model_name, model in tqdm(Models):
    Scores[model_name] = cross_val_score(model, x_train, y_train, cv=10, scoring='accuracy')

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
dt = pd.DataFrame.from_dict(Scores)
ax = sns.boxplot(data=dt, ax=ax)
for m, s in Scores.items():
    print(m, list(s)[:4])

# <center><font color="blue">Hyperparameter Optimization</font></center>

<ul>
<li>Misal akan dicontohkan dua algoritma (model) yang sudah kita bahas sebelumnya: k-NN dan SVM</li>
<li>Sebagai latihan silahkan untuk mencoba HO pada model yang lain.</li>
<li>Parameter tiap model di ML berbeda-beda dan nilai optimalnya berbeda pada setiap kasus.</li>
</ul>
<p><img src="images/rand_grid_search.png" alt="" width="811" height="406" /></p>

<img alt="" src="images/grid_search_workflow.png" />

In [ ]:
# Hyperparameter optimization pada model kNN menggunakan gridCV
kCV = 10
metric = 'accuracy'
params = {}
params['kneighborsclassifier__n_neighbors'] = [1, 3, 5, 10, 15, 20, 25, 30]
params['kneighborsclassifier__weights'] = ('distance', 'uniform')

pipe = make_pipeline(neighbors.KNeighborsClassifier())
optKnn = GridSearchCV(pipe, params, cv=kCV, scoring=metric, verbose=1, n_jobs=-2) #
optKnn.fit(x_train, y_train)
print(optKnn.best_score_)
print(optKnn.best_params_)

In [ ]:
# Contoh Hyperparameter optimization pada model SVM menggunakan RandomizedSearchCV
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
# Berikut ini contoh bagaimana mengetahui parameter yang dapat kita optimasi.
# Gunakan pengetahuan teori/analitik untuk mengoptimasi hanya parameter yang paling penting.
pipeSVM = make_pipeline(svm.SVC())
print(sorted(pipeSVM.get_params().keys()))

In [ ]:
# Optimal parameter SVM dengan RandomizedSearch
# WARNING cell ini butuh waktu komputasi cukup lama
kCV = 10
paramsSVM = {}
paramsSVM['svc__C'] = [1, 10, 100, 1000] #sp.stats.uniform(scale=100)
paramsSVM['svc__gamma'] = [0.1, 0.001, 0.0001, 1, 10]
paramsSVM['svc__kernel'] = ['rbf', 'sigmoid', 'linear'] # , 'poly'
optSvm = RandomizedSearchCV(pipeSVM, paramsSVM, cv=kCV, scoring=metric, verbose=2, n_jobs=-2) # refit=True, pre_dispatch='2*n_jobs' pre_dispatch min 2* n_jobs
optSvm.fit(x_train, y_train)
print(optSvm.best_score_)
print(optSvm.best_params_)

# Model Selection

<img alt="" src="images/model_selection.png" />

In [ ]:
kCV = 10
# Menggunakan parameter optimal
kNN = neighbors.KNeighborsClassifier(n_neighbors= 20, weights= 'uniform')
svm_ = svm.SVC(kernel= 'linear', gamma= 10, C= 10)

# Melakukan Cross Validasi
models = ['kNN', 'SVM']
knn_score = cross_val_score(kNN, x_train, y_train, cv=kCV, scoring='accuracy', n_jobs=-2, verbose=1)
svm_score = cross_val_score(svm_, x_train, y_train, cv=kCV, scoring='accuracy', n_jobs=-2, verbose=1)
scores = [knn_score, svm_score]

data = {m:s for m,s in zip(models, scores)}
for name in data.keys():
    print("Accuracy %s: %0.2f (+/- %0.2f)" % (name, data[name].mean(), data[name].std() * 2))

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
p = sns.boxplot(data=pd.DataFrame(data), ax=ax)

# <center><font color="blue">Ensemble Model</font></center>

<ul>
	<li>What? a learning algorithms that construct a set of classifiers and then classify new data points by taking a (weighted) vote of their predictions.</li>
	<li>Why? Better prediction, More stable model</li>
	<li>How? Bagging &amp; Boosting</li>
</ul>
<img alt="" src="images/Ensemble.png" style="width: 500px; height: 213px;" />

# <center><font color="blue">“meta-algorithms” : Bagging & Boosting</font></center>

<p><img style="undefined: undefined;" src="images/Bagging_VS_Boosting.png" alt="" width="911" height="337" /></p>

* Ensemble https://www.youtube.com/watch?v=Un9zObFjBH0 
* Bagging https://www.youtube.com/watch?v=2Mg8QD0F1dQ 
* Boosting https://www.youtube.com/watch?v=GM3CDQfQ4sw

# <center><font color="blue">Boosting in ML</font></center>

<center><video controls src="videos/Boosting.mp4"/></center>

# <center><font color="blue">Property of Boosting</font></center>
<img alt="" src="images/Bagging-Boosting_Usage.png" style="width: 500px; height: 281px;" />

# <center><font color="blue">AdaBoost</font></center>
<ul>
	<li><a href="https://youtu.be/BoGNyWW9-mE?t=70" target="_blank">https://youtu.be/BoGNyWW9-mE?t=70</a></li>
</ul>
<img alt="" src="images/AdaBoost.png" style="width: 400px; height: 332px;" />

In [ ]:
# Contoh Voting (Bagging) di Python
# Catatan : Random Forest termasuk Bagging Ensemble (walau modified)
# Best practicenya Model yang di ensemble semuanya menggunakan Optimal Parameter

kNN = neighbors.KNeighborsClassifier(3)
kNN.fit(X_train, Y_train)
Y_kNN = kNN.score(X_test, Y_test)

DT = tree.DecisionTreeClassifier(random_state=1)
DT.fit(X_train, Y_train)
Y_DT = DT.score(X_test, Y_test)

model = VotingClassifier(estimators=[('k-NN', kNN), ('Decision Tree', DT)], voting='hard')
model.fit(X_train,Y_train)
Y_Vot = model.score(X_test,Y_test)

print('Akurasi k-NN', Y_kNN)
print('Akurasi Decision Tree', Y_DT)
print('Akurasi Votting', Y_Vot)

In [ ]:
# Averaging juga bisa digunakan di Klasifikasi (ndak hanya Regresi), 
# tapi kita pakai probabilitas dari setiap kategori
T = tree.DecisionTreeClassifier()
K = neighbors.KNeighborsClassifier()
R = LogisticRegression()

T.fit(X_train,Y_train)
K.fit(X_train,Y_train)
R.fit(X_train,Y_train)

y_T=T.predict_proba(X_test)
y_K=K.predict_proba(X_test)
y_R=R.predict_proba(X_test)

Ave = (y_T+y_K+y_R)/3
print(Ave[:5]) # Print just first 5
prediction = [v.index(max(v)) for v in Ave.tolist()]
print(prediction[:5]) # Print just first 5
print('Akurasi Averaging', accuracy_score(Y_test, prediction))

In [ ]:
# AdaBoost
num_trees = 100
kfold = model_selection.KFold(n_splits=10)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=33)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

# <center><font color="blue">Imbalance Data</font></center>

* Metric Trap
* Akurasi kategori tertentu lebih penting
* Contoh kasus
<img alt="" src="images/imbalance.png" />

# <center><font color="blue">Imbalance Learning</font></center>

<img alt="" src="images/under-over-sampling.png" style="width: 500px; height: 147px;" />

* Undersampling, Oversampling, Model Based (weight adjustment)
* https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets
* Plot perbandingan: https://imbalanced-learn.readthedocs.io/en/stable/auto_examples/combine/plot_comparison_combine.html#sphx-glr-auto-examples-combine-plot-comparison-combine-py

In [ ]:
data.head()

In [ ]:
p = data["class"].value_counts().plot(kind='pie')

In [ ]:
# fit the model and get the separating hyperplane using weighted classes

svm_balanced = svm.SVC(kernel='linear') #WEIGHTED SVM
svm_balanced.fit(x_train, y_train)
y_SVMb = svm_balanced.predict(x_test)

print(confusion_matrix(y_test, y_SVMb))
print(classification_report(y_test, y_SVMb))

In [ ]:
# fit the model and get the separating hyperplane using weighted classes
# x_train, x_test, y_train, y_test

svm_balanced = svm.SVC(kernel='linear', class_weight={1: 3}) #WEIGHTED SVM
svm_balanced.fit(x_train, y_train)
y_SVMb = svm_balanced.predict(x_test)

print(confusion_matrix(y_test, y_SVMb))
print(classification_report(y_test, y_SVMb))

In [ ]:
# Example of model-based imbalance treatment - SVM
from sklearn.datasets import make_blobs
n_samples_1, n_samples_2 = 1000, 100
centers = [[0.0, 0.0], [2.0, 2.0]]
clusters_std = [1.5, 0.5]
X, y = make_blobs(n_samples=[n_samples_1, n_samples_2],centers=centers,cluster_std=clusters_std,random_state=33, shuffle=False)

# fit the model and get the separating hyperplane
clf = svm.SVC(kernel='linear', C=1.0)
clf.fit(X, y)

# fit the model and get the separating hyperplane using weighted classes
wclf = svm.SVC(kernel='linear', class_weight={1: 10}) #WEIGHTED SVM
wclf.fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, edgecolors='k')# plot the samples
ax = plt.gca()# plot the decision functions for both classifiers
xlim = ax.get_xlim(); ylim = ax.get_ylim()
xx = np.linspace(xlim[0], xlim[1], 30)# create grid to evaluate model
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)# get the separating hyperplane
a = ax.contour(XX, YY, Z, colors='k', levels=[0], alpha=0.5, linestyles=['-']) # plot decision boundary and margins
Z = wclf.decision_function(xy).reshape(XX.shape)# get the separating hyperplane for weighted classes
b = ax.contour(XX, YY, Z, colors='r', levels=[0], alpha=0.5, linestyles=['-'])# plot decision boundary and margins for weighted classes
plt.legend([a.collections[0], b.collections[0]], ["non weighted", "weighted"], loc="upper right")
plt.show()

# <center><font color="blue">Weighted Decision Tree</font></center>

In [ ]:
T = tree.DecisionTreeClassifier(random_state = 33)
T.fit(X_train,Y_train)
y_DT = T.predict(X_test)
print('Akurasi  (Decision tree Biasa) = ', accuracy_score(Y_test, y_DT))
print(classification_report(Y_test, y_DT))

T = tree.DecisionTreeClassifier(class_weight = 'balanced', random_state = 33)
T.fit(X_train,Y_train)
y_DT = T.predict(X_test)
print('Akurasi  (Weighted Decision tree) = ', accuracy_score(Y_test, y_DT))
print(classification_report(Y_test, y_DT))

# <center><font color="blue"> Studi Kasus (Latihan) ENB2012: Prediksi Penggunaan Energi Gedung</font></center>

<img src="images/buildings-by-calibrated-simulation.png" alt="" width="503" height="377" />

# <font color="green"> Task</font></center>

* Filter data EcoTest dan pilih hanya yang kategori di variabel target muncul min 10 kali (heat-cat)
* Lakukan EDA (Preprocessing dan visualisasi dasar)
* Tentukan model terbaik (dengan parameter optimal dan cross validasi)
* Hati-hati Naive Bayes, Decision Tree dan Random Forest tidak memerlukan one-hot encoding.
* Gunakan Metric Micro F1-Score untuk menentukan model terbaiknya.

# <font color="green">Optional</font></center>
* Coba bandingkan model terbaik diatas dengan model ensemble.
* Apakah ada imbalance problem, coba atasi dengan over/under sampling.

In [ ]:
file_ = "data/building-energy-efficiency-ENB2012_data.csv"

try: # Running Locally, yakinkan "file_" berada di folder "data"
    data = pd.read_csv(file_, error_bad_lines=False, low_memory = False, encoding='utf8')
except: # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/ptpjb/master/{file_}
    data = pd.read_csv(file_, error_bad_lines=False, low_memory = False, encoding='utf8')
print(data.shape)
data.sample(5)

In [ ]:
# Jawaban Latihan dimulai di cell ini


# <center><font color="blue"> Akhir Modul 12 - Model Klasifikasi Lanjutan</font></center>

<hr />
<img alt="" src="images/meme-cartoon/Meme Deep Learning SVM.jpg" style="height: 400px;"/>